# 信号処理ライブラリを使用する場合/With a signal processing library

In [ ]:
import librosa
import numpy as np
from scipy import signal
import IPython.display
%matplotlib inline
import matplotlib.pyplot as plt
import os

In [ ]:
# オリジナル音源データの読み込み/Load original sound data

if not os.path.isdir('fukuzatu_jikken'):
  !git clone https://github.com/shinolab/fukuzatu_jikken.git -b data

fs = 44100
orig_wav, _ = librosa.load('./fukuzatu_jikken/Hallelujah.mp3', sr=fs, offset = 7.0, duration=5.0)
IPython.display.Audio(data=orig_wav, rate=fs)

In [ ]:
# ホワイトノイズから低周波/高周波ノイズを作成/Generate high/low frequency noise from white noise 

high_noise_f = 10000.0
low_noise_f = 400.0
fir_low_pass = signal.firwin(4095, low_noise_f, fs = fs) # Low-pass filter
fir_high_pass = signal.firwin(4095, high_noise_f, fs = fs, pass_zero=False) # High-pass filter
white_noise = np.random.rand(round(fs*5.0))

low_noise = signal.lfilter(fir_low_pass, 1, white_noise)
low_noise = low_noise - np.mean(low_noise)
low_noise = low_noise / np.max(np.abs(low_noise))
high_noise = signal.lfilter(fir_high_pass, 1, white_noise)
high_noise = high_noise - np.mean(high_noise)
high_noise = high_noise / np.max(np.abs(high_noise))

In [ ]:
IPython.display.Audio(data=low_noise, rate=fs)

In [ ]:
IPython.display.Audio(data=high_noise, rate=fs)

In [ ]:
N = len(low_noise)
dt = 1/fs
freq = np.fft.fftfreq(N, dt)[:N//2]
plt.plot(freq, 2.0 / N * np.abs(np.fft.fft(low_noise)[:N//2]))
plt.xscale('log')

In [ ]:
N = len(high_noise)
dt = 1/fs
freq = np.fft.fftfreq(N, dt)[:N//2]
plt.plot(freq, 2.0 / N * np.abs(np.fft.fft(high_noise)[:N//2]))
plt.xscale('log')

In [ ]:
mix_wave = orig_wav + 0.6 * low_noise + 0.15* high_noise
IPython.display.Audio(data=mix_wave, rate=fs)

In [ ]:
fir_low_pass = signal.firwin(4095, high_noise_f/2, fs = fs)
fir_high_pass = signal.firwin(4095, low_noise_f, fs = fs, pass_zero=False)
filtered_wave = signal.lfilter(fir_low_pass, 1, mix_wave)
filtered_wave = signal.lfilter(fir_high_pass, 1, filtered_wave)

In [ ]:
IPython.display.Audio(data=filtered_wave, rate=fs)

In [ ]:
N = len(filtered_wave)
dt = 1/fs
freq = np.fft.fftfreq(N, dt)[:N//2]
plt.plot(freq, 2.0 / N * np.abs(np.fft.fft(filtered_wave)[:N//2]))
plt.xscale('log')